# TorchMD Simulation

This notebook runs a short torchmd simulation of a water box.

In [1]:
#SBATCH -p gpu --exclusive --time 24:00:00
import sys
sys.path.append(".")
DEVICE = "cpu"
#DEVICE = "cuda:0"

In [2]:
import torch
import numpy as np

Set seeds to ensure reproducibility.

In [3]:
torch.manual_seed(1)
np.random.seed(1)

## Build Test System

The files defining the water box are part of the `torchmd` test suite. The setup is defined in the module `waterbox.py`.
The box contains 96 water molecules as well as one ion pair (sodium and chlorine).

In [5]:
from waterbox import WaterBox
waterbox = WaterBox(nreplicas=1, device=DEVICE)

## Energy Minimization

In [6]:
from torchmd.minimizers import minimize_bfgs
minimize_bfgs(waterbox.system, waterbox.forces, steps=500)  # Minimize the system

Iter  Epot            fmax    
   0   -691.583104    57.640355
   1   -249.736811    212.411990
   2   -731.377728    19.837357
   3   -740.159184    20.139690
   4   -786.285572    43.255250
   5   -792.541717    66.349296
   6   -815.886302    22.912048
   7   -830.756328    21.255851
   8   -842.972508    26.154271
   9   -852.190711    18.151096
  10   -857.584258    92.193282
  11   -871.894552    28.030490
  12   -882.560370    23.991500
  13   -893.327520    46.935104
  14   -897.904242    31.250359
  15   -903.884071    12.794050
  16   -910.240079    20.790360
  17   -917.728849    23.770905
  18   -927.771507    25.981902
  19   -933.377868    28.852858
  20   -941.155280    16.347461
  21   -945.924788    15.937664
  22   -951.955510    17.664942
  23   -957.618717    48.166648
  24   -968.034143    20.388571
  25   -975.991177    19.592387
  26   -982.013636    14.397089
  27   -984.929808    30.084177
  28   -990.443738    14.777269
  29   -996.866784    15.398846
  30   -

 255   -1223.516115    7.646166
 256   -1223.647512    4.009548
 257   -1223.768407    2.360777
 258   -1223.892486    2.195725
 259   -1223.958145    2.436329
 260   -1224.029577    1.734739
 261   -1224.115809    1.359326
 262   -1224.173308    1.883137
 263   -1224.255659    2.238765
 264   -1224.319040    1.981369
 265   -1224.369152    1.144486
 266   -1224.404465    1.235313
 267   -1224.466272    1.774222
 268   -1224.532681    2.281679
 269   -1224.607714    1.550812
 270   -1224.677193    1.465915
 271   -1224.738718    1.466345
 272   -1224.845911    5.314904
 273   -1224.976853    1.718935
 274   -1225.081539    2.059790
 275   -1225.226841    3.535013
 276   -1225.365922    4.378391
 277   -1225.535999    2.493423
 278   -1225.729330    4.029442
 279   -1225.879688    6.159706
 280   -1226.037648    3.328875
 281   -1226.267294    3.233387
 282   -1226.423692    4.446878
 283   -1226.602420    2.740390
 284   -1226.749637    2.729400
 285   -1226.870848    4.528869
 286   -

## Prepare Simulation

In [7]:
from torchmd.integrator import Integrator
from torchmd.wrapper import Wrapper

langevin_temperature = 300.0  # K
langevin_gamma = 10
timestep = 1  # fs

integrator = Integrator(
    waterbox.system, 
    waterbox.forces, 
    timestep, 
    waterbox.device, 
    gamma=langevin_gamma, 
    T=langevin_temperature
)
wrapper = Wrapper(
    waterbox.mol.numAtoms, 
    waterbox.mol.bonds if len(waterbox.mol.bonds) else None, 
    waterbox.device
)

In [8]:
from torchmd.utils import LogWriter

logger = LogWriter(
    path="logs/",
    keys=('iter','ns','epot','ekin','etot','T'),
    name='monitor.csv'
)

Writing logs to  logs/monitor.csv


## Equilibration

In [10]:
*_, T = integrator.step(niter=100)
*_, T = integrator.step(niter=100)
print(T)

[293.92907073]


## Run NVE Simulation

In [16]:
integrator = Integrator(
    waterbox.system, 
    waterbox.forces, 
    timestep, 
    waterbox.device, 
    gamma=0.0, 
    T=None
)

In [17]:
from tqdm import tqdm 
import numpy as np

FS2NS = 1E-6 # Femtosecond to nanosecond conversion

steps = 10000
output_period = 10
save_period = 100
traj = []

system = waterbox.system
forces = waterbox.forces

trajectoryout = "xyz_vel.npy"
iterator = tqdm(range(1, int(steps / output_period) + 1))
Epot = forces.compute(system.pos, waterbox.system.box, system.forces)
for i in iterator:
    Ekin, Epot, T = integrator.step(niter=output_period)
    wrapper.wrap(system.pos, system.box)
    currpos = system.pos.detach().cpu().numpy().copy()
    currvel = system.vel.detach().cpu().numpy().copy()
    traj.append(np.stack([currpos[0], currvel[0]], axis=0))
    
    if (i*output_period) % save_period  == 0:
        np.save(trajectoryout, np.stack(traj, axis=0))

    logger.write_row({
        'iter':i*output_period,
        'ns':FS2NS*i*output_period*timestep,
        'epot':Epot,
        'ekin':Ekin,
        'etot':Epot+Ekin,
        'T':T
    })


  0%|          | 1/1000 [00:00<06:37,  2.52it/s]

[265.20263919]



  0%|          | 2/1000 [00:00<06:30,  2.55it/s]

[270.27728515]



  0%|          | 3/1000 [00:01<06:29,  2.56it/s]

...

[283.49447781]



  2%|▏         | 21/1000 [00:08<06:31,  2.50it/s]

[286.38086999]


  2%|▏         | 21/1000 [00:08<06:41,  2.44it/s]


KeyboardInterrupt: 